In [401]:
import numpy as np
import random

## Trainingsdaten und DNF

In [402]:
dnf_training = np.array([[-1, -1, -1,  1, -1,  1, -1,  1,  0, -1],
                        [-1,  0, -1,  1,  0, -1,  1,  1, -1,  0],
                        [ 1, -1,  1,  0, -1, -1, -1,  1,  0,  0],
                        [ 0, -1,  1,  0, -1, -1, -1,  1, -1, -1],
                        [-1, -1,  0,  1, -1,  1,  1,  0,  0, -1]]).astype(np.float64)


inputs_with_p = [
    (np.array([-1, -1, -1,  1, -1,  1, -1,  1,  1, -1]), 1),
    (np.array([-1,  1, -1,  1,  1, -1,  1,  1, -1,  1]), 1),
    (np.array([ 1, -1,  1,  1, -1, -1, -1,  1,  1,  1]), 1),
    (np.array([ 1, -1,  1,  1, -1, -1, -1,  1, -1, -1]), 1),
    (np.array([-1, -1,  1,  1, -1,  1,  1,  1,  1, -1]), 1),
    (np.array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1]), -1),
    (np.array([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1]), -1),
    (np.array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0]), -1),
    (np.array([ 1, -1,  1, -1,  1, -1,  1, -1,  1, -1]), -1),
    (np.array([-1,  1, -1,  1, -1,  1, -1,  1, -1,  1]), -1)
]

## Aktivierungsfunktion (Signum-Funktion)

In [403]:
# Signum-Funktion (sig), die die Werte eines Vektors auf -1 oder 1 setzt
def sig(x):
    """
    Wendet die Signum-Funktion auf jeden Wert eines Vektors an.
    
    Parameter:
    - x: Eingabevektor (numpy-Array)
    
    Rückgabe:
    - Ein numpy-Array mit Werten -1 oder 1.
    """
    ret = np.copy(x)  # Kopiere den Eingabevektor
    for i in range(np.shape(x)[0]):  # Iteriere über alle Elemente
        if x[i] >= 0:
            ret[i] = 1  # Setze positive Werte auf 1
        else:
            ret[i] = -1  # Setze negative Werte auf -1
    return ret.astype(np.int64)  # Rückgabe als Integer-Array

## Hilfsfunktion

In [404]:
# Funktion zur Berechnung der Anzahl der verwendeten Literale in einem Monom
def get_used_literals(monome):
    """
    Zählt die Anzahl der nicht-null Werte in einem Monom.
    
    Parameter:
    - monome: Ein Array, das ein Monom repräsentiert
    
    Rückgabe:
    - Anzahl der verwendeten Literale (Werte ungleich 0).
    """
    literal_count = 0
    for elem in monome:
        if elem != 0:
            literal_count += 1  # Erhöhe den Zähler für jedes nicht-null Element
    return literal_count

## Schwellenwert zur Berechnung des Z-Layers

In [405]:
def calculate_treshold_z(w):

    v = np.empty(w.shape[0], dtype=np.float64)
    for i in range(w.shape[0]):
        v[i] = get_used_literals(w[i]) - 0.5
    
    return v

## Berechnung z-Layer

In [406]:
def calculate_z(w, input, v):

    # Berechnung der z-Werte
    z = np.empty(w.shape[0], dtype=np.float64)  # Initialisiere z
    for j in range(w.shape[0]):
        z[j] = np.dot(w[j], input) - v[j]  # Skalarprodukt minus v[j]
    z = sig(z)  # Wende die Signum-Funktion auf z an

    return z

## Berechnung der Schwellenwerte und Gewichte für das y-Layer

In [407]:
def calculate_treshold_and_weights_y(z):

    W = np.ones((z), np.float64)
    V = -(z - 1)

    return W, V

## Berechnung des y-Layer

In [408]:
def calculate_y(W, V, z):

    y = np.matmul(W, z.T)

    y = y - V
    if y >= 0:
        y = 1
    else:
        y = -1
    return y

## Test

In [409]:
input = inputs_with_p[0][0]  # Beispiel-Eingabevektor
print("Input:", input)

# Berechnung der Gewichtsmatrix W und der Schwellenwerte v
z_treshhold = calculate_treshold_z(dnf_training)
print("Tresholds:", z_treshhold)
#Berechne z
z = calculate_z(dnf_training, input, z_treshhold)
print("Z:", z)

#Berechnung der Gewichtsmatrix W und des Schwellenwerts V für die Ausgabe
y_treshhold = calculate_treshold_and_weights_y(z.shape[0])
print("W: ", y_treshhold[0])
print("V: ", y_treshhold[1])

# Berechne y
y = calculate_y(y_treshhold[0], y_treshhold[1], z)
print("y:", y)

Input: [-1 -1 -1  1 -1  1 -1  1  1 -1]
Tresholds: [8.5 6.5 6.5 7.5 6.5]
Z: [ 1 -1 -1 -1 -1]
W:  [1. 1. 1. 1. 1.]
V:  -4
y: 1


## Backpropagation

In [410]:
def calculate_deltas(input, p, lernrate, z, y, w, v, W, V):
    
    delta_w = np.copy(w)
    delta_v = np.copy(v)
    delta_W = np.copy(W)
    delta_V = np.copy(V)

    error = p - y

    for j in range(W.size):
        delta_W[j] = lernrate * error * z[j]
        delta_v[j] = -lernrate * error * W[j]
        for k in range(input.size):
            delta_w[j][k] = lernrate * W[j] * error * input[k]
    delta_V = -lernrate * error

    return delta_w, delta_v, delta_W, delta_V

In [411]:
""" # Test Backpropagation
p = inputs_with_p[0][1]  # Beispiel-Output
print(calculate_deltas(input, p, 0.1, z, y, dnf_training, z_treshhold, y_treshhold[0], y_treshhold[1])) """

' # Test Backpropagation\np = inputs_with_p[0][1]  # Beispiel-Output\nprint(calculate_deltas(input, p, 0.1, z, y, dnf_training, z_treshhold, y_treshhold[0], y_treshhold[1])) '

In [ ]:
def change_weights(dnf, full):

        
    for i in range(dnf.shape[0]):
        #print(i)
        for j in range(dnf.shape[1]):
            #print(j)
            if full == False:
                dnf[i][j] += (random.randint(-10 , 10) * 0.1)
            else:
                dnf[i][j] = random.randint(-10, 10) * 0.1
    

    return dnf

In [419]:
def feedforward_backpropagation(dnf, trainings_data, lerningsrate, epochs):
    
    w = change_weights(dnf, True)
    v = calculate_treshold_z(w)
    y_treshhold = calculate_treshold_and_weights_y(w.shape[0])
    W, V = y_treshhold
    
    for epoch in range(epochs):
        random_int = random.randint(0, len(trainings_data) - 1)
        input = trainings_data[random_int][0]
        p = trainings_data[random_int][1]
        z = calculate_z(w, input, v)
        y  = calculate_y(W, V, z)

        """     
        print("_______________________________________________________________")
        print("Epoch:", epoch + 1)
        print("Input:", input)
        print("w = ", w)
        print("v = ", v)
        print("z = ", z)
        print("W = ", y_treshhold[0])
        print("V = ", y_treshhold[1])
        print("y = ", y)
        print("p = ", p)
        print("Fehler = ", p - y)
        print("_______________________________________________________________")
        """
        print(f"y= {y}, p= {p} Fehler={p - y}")

        delta_w, delta_v, delta_W, delta_V = calculate_deltas(input, p, lerningsrate, z, y, w, v, y_treshhold[0], y_treshhold[1])
        w = np.add(w, delta_w)
        v = np.add(v, delta_v)
        W = np.add(y_treshhold[0], delta_W)
        V = np.add(y_treshhold[1], delta_V)

        

In [420]:
# feedforward_backpropagation Test

""" v = calculate_treshold_z(dnf_training)
z = calculate_z(dnf_training, input, v)
W, V = calculate_treshold_and_weights_y(z.shape[0])
y = calculate_y(W, V, z) """


feedforward_backpropagation(dnf_training, inputs_with_p, 0.1, 100)

NameError: name 'Flase' is not defined

In [415]:
print(change_weights(dnf_training.astype(np.float64)))

[[-1.40000000e+00 -9.00000000e-01 -2.90000000e+00 -4.00000000e-01
  -1.40000000e+00 -1.00000000e-01 -8.00000000e-01  1.50000000e+00
   1.10000000e+00 -2.90000000e+00]
 [-1.70000000e+00 -1.10000000e+00 -5.00000000e-01  7.00000000e-01
   1.00000000e-01 -2.00000000e-01  6.00000000e-01  2.00000000e-01
  -2.40000000e+00 -1.50000000e+00]
 [ 2.90000000e+00 -9.00000000e-01  1.00000000e-01  1.40000000e+00
   5.55111512e-17 -1.90000000e+00 -1.20000000e+00  7.00000000e-01
   9.00000000e-01  1.40000000e+00]
 [-3.00000000e-01 -4.00000000e-01  8.00000000e-01 -1.10000000e+00
  -3.00000000e-01 -2.60000000e+00 -1.10000000e+00  1.40000000e+00
  -1.30000000e+00 -1.50000000e+00]
 [-1.00000000e+00  3.00000000e-01  7.00000000e-01  1.40000000e+00
  -1.60000000e+00 -1.11022302e-16  1.10000000e+00 -2.00000000e-01
  -4.00000000e-01 -1.50000000e+00]]
